In [1]:
import pandas as pd
from shapely.geometry import Polygon, Point,box
import geopandas as gpd
from pyproj import Transformer, Geod

In [13]:
data = pd.read_csv("lausanne_migration_2011_2023.csv", delimiter=",")


In [ ]:
data

,id,X,hectare_id,k_id,name,kanton_name,pop_tot,pop_foreign,pop_swiss,birth_tot,...,immigration_rate,emigration_rate,inmigration_rate,birth_rate,death_rate,year,x_coord,y_coord,outmigration_rate,pop_mean
0,1,12867,122490,CH1551,Lausanne,VD,2554.5,740.5,1814.0,18.0,...,35.231944,21.530632,85.731063,7.046389,15.658642,2011-2014,2537850,1151750,NaN,638.625
1,2,12868,835618,CH1551,Lausanne,VD,3083.0,1132.0,1951.0,24.0,...,11.028219,7.460266,96.334739,7.784625,1.946156,2011-2014,2539050,1154650,NaN,770.750
2,3,12869,717249,CH1551,Lausanne,VD,1914.5,1056.0,858.5,18.0,...,59.023244,38.652390,80.961086,9.401933,3.656307,2011-2014,2535850,1152650,NaN,478.625
3,4,12870,973191,CH1551,Lausanne,VD,1795.5,434.0,1361.5,15.0,...,6.126427,11.138959,60.707324,8.354219,8.354219,2011-2014,2539650,1153850,NaN,448.875
4,5,12871,484438,CH1551,Lausanne,VD,1992.0,759.5,1232.5,18.0,...,18.574297,24.096386,74.297189,9.036145,7.028112,2011-2014,2539750,1153350,NaN,498.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,1001,56278,1372450,CH1551,Lausanne,VD,1900.0,815.5,1084.5,46.0,...,10.526316,9.473684,57.368421,24.210526,3.157895,2020-2023,2538850,1153250,NaN,475.000
1001,1002,56279,318754,CH1551,Lausanne,VD,1361.5,576.5,785.0,33.0,...,11.751744,8.079324,58.024238,24.237973,NaN,2020-2023,2539850,1154450,NaN,340.375
1002,1003,56280,700234,CH1551,Lausanne,VD,2091.5,1344.0,747.5,37.0,...,69.806359,42.075066,207.028448,17.690653,NaN,2020-2023,2536750,1152850,NaN,522.875
1003,1004,56281,608314,CH1551,Lausanne,VD,1045.5,591.5,454.0,23.0,...,50.693448,37.302726,124.342420,21.999044,NaN,2020-2023,2539650,1154550,NaN,261.375


In [15]:
# Initialize transformers
transformer_to_wgs84 = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)
geod = Geod(ellps="WGS84")


In [16]:
lon, lat = transformer_to_wgs84.transform(
    data["x_coord"].values, data["y_coord"].values
)

In [17]:
def create_square(lat, lon, size_m=100):
    """
    Create a square polygon around a point (lat, lon) with sides of length size_m meters.
    """
    # Calculate half the size
    half_size = size_m / 2.0

    # Calculate the coordinates of the square's corners
    # Starting from the center point, calculate the corners in N-E-S-W directions
    # Note: Azimuths are calculated clockwise from north: 0° (N), 90° (E), 180° (S), 270° (W)
    points = []

    # Calculate corner points
    # Starting from the center, move to each corner
    for angle in [45, 135, 225, 315]:
        dest_lon, dest_lat, _ = geod.fwd(
            lon, lat, angle, half_size * 1.4142
        )  # Diagonal distance
        points.append((dest_lon, dest_lat))

    # Close the polygon
    points.append(points[0])

    return Polygon(points)

In [23]:

def create_square2(lat, lon, size_m=100):


    # Create GeoDataFrame with a Point at (lon, lat)
    gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat)], crs="EPSG:4326")

    # Convert to a projection that uses meters
    gdf = gdf.to_crs("EPSG:3857")

    gdf['geometry'] = gdf['geometry'].apply(lambda x: x.buffer(size_m/2).envelope)
    gdf = gdf.to_crs({'init': 'epsg:4326'})

    return gdf.iloc[0].geometry


In [24]:
# Generate polygons
polygons = [create_square(lat_i, lon_i, size_m=100) for lat_i, lon_i in zip(lat, lon)]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry=polygons, crs="EPSG:4326")

In [25]:
gdf.to_file("lausanne_migration_2011_2023.geojson", driver="GeoJSON")
